In [1]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import LSTM,Input,Bidirectional,Dense
import numpy as np

batch_size=64
epochs=100
latent_dim=256
num_samples=10000
data_path='./tam.txt'

In [2]:
input_texts=[]
target_texts=[]
input_characters=set()
target_characters=set()

with open(data_path,'r',encoding='utf-8') as f:
    lines=f.read().split("\n")

for line in lines[: min(num_samples,len(lines)-1)]:
    input_text,target_text,_=line.split("\t")
    
    target_text='\t'+target_text+'\n'
    
    input_texts.append(input_text)
    target_texts.append(target_text)
    
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
            
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

In [3]:
input_texts

['I slept.',
 'Calm down.',
 "I'll walk.",
 'Who is he?',
 'Who knows?',
 'She smiled.',
 'Talk to me!',
 'Who is she?',
 'Go to sleep.',
 'It may rain.',
 'She bit him.',
 'She hit him.',
 'She is kind.',
 'She is eight.',
 'Where are we?',
 'Keep in touch!',
 'See you again.',
 'Give it to her.',
 'I ate too much.',
 "I'll see to it.",
 "It's up to you.",
 'Leave it to me.',
 'Listen to this!',
 "That's the way.",
 'Come and see me.',
 "Don't lie to me.",
 'He began to run.',
 'He just arrived.',
 'He likes to run.',
 'How is your dad?',
 'I want to sleep.',
 "I'm able to run.",
 'Raise your hand.',
 'What did he say?',
 'When can we eat?',
 'Come and help us.',
 'He is still here.',
 'I have to go now.',
 'I know that much.',
 'I made a mistake.',
 'I walk to school.',
 "That's our house.",
 'Those are my CDs.',
 'Walk ahead of me.',
 "We'll follow you.",
 'Beware of the dog!',
 'He came back soon.',
 'He has three sons.',
 'I know how to ski.',
 'I know what to do.',
 "I'm kind of 

In [4]:
target_characters

{'\t',
 '\n',
 ' ',
 '!',
 '(',
 ')',
 ',',
 '.',
 '0',
 '2',
 '?',
 'C',
 'D',
 'அ',
 'ஆ',
 'இ',
 'உ',
 'ஊ',
 'எ',
 'ஏ',
 'ஒ',
 'ஓ',
 'க',
 'ங',
 'ச',
 'ஜ',
 'ஞ',
 'ட',
 'ண',
 'த',
 'ந',
 'ன',
 'ப',
 'ம',
 'ய',
 'ர',
 'ற',
 'ல',
 'ள',
 'ழ',
 'வ',
 'ஷ',
 'ஸ',
 'ா',
 'ி',
 'ீ',
 'ு',
 'ூ',
 'ெ',
 'ே',
 'ை',
 'ொ',
 'ோ',
 '்'}

In [5]:
input_characters=sorted(list(input_characters))
target_characters=sorted(list(target_characters))
num_encoder_token=len(input_characters)
num_target_token=len(target_characters)
max_encoder_length=max([len(text) for text in input_texts])
max_decoder_length=max([len(text) for text in target_texts])

In [6]:
print("max_encoder_length",max_encoder_length)
print("max_decoder_length",max_decoder_length)
print("unique encoder",num_encoder_token)
print("unique decoder",num_target_token)

max_encoder_length 94
max_decoder_length 111
unique encoder 53
unique decoder 54


In [7]:
input_text

"It's my fault that the cake was burned. I was talking on the phone and didn't notice the time."

In [8]:
input_token_index=dict([(i,char) for char,i in enumerate(input_characters)])
target_token_index=dict([(i,char) for char,i in enumerate(target_characters)])

In [9]:
input_token_index,target_token_index

({' ': 0,
  '!': 1,
  "'": 2,
  ',': 3,
  '.': 4,
  '0': 5,
  '2': 6,
  '?': 7,
  'A': 8,
  'B': 9,
  'C': 10,
  'D': 11,
  'F': 12,
  'G': 13,
  'H': 14,
  'I': 15,
  'J': 16,
  'K': 17,
  'L': 18,
  'M': 19,
  'N': 20,
  'P': 21,
  'R': 22,
  'S': 23,
  'T': 24,
  'W': 25,
  'Y': 26,
  'a': 27,
  'b': 28,
  'c': 29,
  'd': 30,
  'e': 31,
  'f': 32,
  'g': 33,
  'h': 34,
  'i': 35,
  'j': 36,
  'k': 37,
  'l': 38,
  'm': 39,
  'n': 40,
  'o': 41,
  'p': 42,
  'q': 43,
  'r': 44,
  's': 45,
  't': 46,
  'u': 47,
  'v': 48,
  'w': 49,
  'x': 50,
  'y': 51,
  'z': 52},
 {'\t': 0,
  '\n': 1,
  ' ': 2,
  '!': 3,
  '(': 4,
  ')': 5,
  ',': 6,
  '.': 7,
  '0': 8,
  '2': 9,
  '?': 10,
  'C': 11,
  'D': 12,
  'அ': 13,
  'ஆ': 14,
  'இ': 15,
  'உ': 16,
  'ஊ': 17,
  'எ': 18,
  'ஏ': 19,
  'ஒ': 20,
  'ஓ': 21,
  'க': 22,
  'ங': 23,
  'ச': 24,
  'ஜ': 25,
  'ஞ': 26,
  'ட': 27,
  'ண': 28,
  'த': 29,
  'ந': 30,
  'ன': 31,
  'ப': 32,
  'ம': 33,
  'ய': 34,
  'ர': 35,
  'ற': 36,
  'ல': 37,
  'ள': 38,
  'ழ'

In [10]:
encoder_input_data=np.zeros((len(input_texts),max_encoder_length,num_encoder_token),dtype="float32")

decoder_input_data=np.zeros((len(target_texts),max_decoder_length,num_target_token),dtype="float32")

decoder_target_data=np.zeros((len(input_texts),max_decoder_length,num_target_token),dtype='float32')

In [11]:
for i,(input_text,target_text) in enumerate(zip(input_texts,target_texts)):
    for t,char in enumerate(input_text):
        encoder_input_data[i,t,input_token_index[char]]=1
    encoder_input_data[i,t+1:,input_token_index[' ']]=1
    for t,char in enumerate(target_text):
        if t>0:
            decoder_target_data[i,t-1,target_token_index[char]]=1
            
        decoder_input_data[i,t+1:,target_token_index[' ']]=1
        decoder_target_data[i,t:,target_token_index[' ']]=1

In [12]:
encoder_input_data[0].shape

(94, 53)

In [13]:
##model building

encoder_inputs=Input(shape=(None,num_encoder_token))
encoder=LSTM(latent_dim,return_state=True)
encoder_outputs,state_h,state_c=encoder(encoder_inputs)

encoder_states=[state_h,state_c]

In [14]:
encoder_states

[<KerasTensor: shape=(None, 256) dtype=float32 (created by layer 'lstm')>,
 <KerasTensor: shape=(None, 256) dtype=float32 (created by layer 'lstm')>]

In [15]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_target_token))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the 
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_target_token, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [16]:
# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',metrics=['accuracy'])
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)

Epoch 1/100


UnknownError:    Fail to find the dnn implementation.
	 [[{{node CudnnRNN}}]]
	 [[model/lstm/PartitionedCall]] [Op:__inference_train_function_5635]

Function call stack:
train_function -> train_function -> train_function


In [ ]:
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

In [ ]:
reverse_input_char_index=dict((i,char) for char,i in input_token_index.items())
reverse_target_char_index=dict((i,char) for char,i in target_token_index.items())

In [ ]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    
    states_value = encoder_model.predict(input_seq)
    
   

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_target_token))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_target_token))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [ ]:
for i in range(100):
    input_seq=encoder_input_data[i: i+1]
    decoded_sentence=decode_sequence(input_seq)
    print("decoded_sentence",decoded_sentence)

In [ ]:
encoder_input_data[0].shape

In [ ]:
decode_sequence(encoder_input_data[2: 2+1])

In [ ]:
import sys
import numpy
numpy.set_printoptions(threshold=sys.maxsize)

In [ ]:
np.array(encoder_input_data[0])

In [ ]:
reverse_target_char_index